http://coloradovoters.info/download.html

In [0]:
## Initialize
import numpy as np
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
## Load data
filename='gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Merged040619.csv'
df = pd.read_csv(filename)

### EDA with 2019 data separately to spot the data architecture
print(df.shape,'\n',df.tail())

(465757, 56) 
          VOTER_ID  COUNTY_CODE  COUNTY     LAST_NAME FIRST_NAME MIDDLE_NAME  \
465752  601970232           16  Denver       JOHNSON       EMMA       GRACE   
465753  601970245           16  Denver         KALIL       ADAM     ABRAXIS   
465754  601970246           16  Denver        KAPLAN    CORINNE     LINDSAY   
465755  601970249           16  Denver  KIMBLE QUINN      BLAKE     ANTHONY   
465756  601970252           16  Denver          KOCH    STEPHEN       DAVID   

       NAME_SUFFIX                VOTER_NAME_LF VOTER_NAME STATUS_CODE ...  \
465752         NaN          JOHNSON, EMMA GRACE        NaN           A ...   
465753         NaN          KALIL, ADAM ABRAXIS        NaN           A ...   
465754         NaN      KAPLAN, CORINNE LINDSAY        NaN           A ...   
465755         NaN  KIMBLE QUINN, BLAKE ANTHONY        NaN           A ...   
465756         NaN          KOCH, STEPHEN DAVID        NaN           A ...   

        BALLOT_RETURNED_DATE  BALLOT_STAG

In [0]:
#Format new address columns
df['Year']=2019
df['Month']=1
df_voter_201901 = df[['VOTER_ID','Year','Month','COUNTY','LAST_NAME','Address','RES_ZIP_CODE','EFFECTIVE_DATE','REGISTRATION_DATE','STATUS','BIRTH_YEAR','GENDER','PRECINCT','SPLIT','PARTY','PREFERENCE','PARTY_AFFILIATION_DATE','CONGRESSIONAL','STATE_SENATE','STATE_HOUSE','COUNCIL_DISTRICT','SCHOOL_DISTRICT','BALLOT_RETURNED_DATE','BALLOT_STAGE','BALLOT_STATUS','IN_PERSON_VOTE','IN_PERSON_VOTE_DATE','VOTED_PARTY']]
df_voter_201901.head()

,VOTER_ID,Year,Month,COUNTY,LAST_NAME,Address,RES_ZIP_CODE,EFFECTIVE_DATE,REGISTRATION_DATE,STATUS,...,STATE_SENATE,STATE_HOUSE,COUNCIL_DISTRICT,SCHOOL_DISTRICT,BALLOT_RETURNED_DATE,BALLOT_STAGE,BALLOT_STATUS,IN_PERSON_VOTE,IN_PERSON_VOTE_DATE,VOTED_PARTY
0,72085,2019,1,Denver,KING,161 S GROVE ST,80219,2/5/2017,7/2/2016,Active,...,34,4,3.0,2.0,10/25/2018,Validated,Accepted,No,NaN,NaN
1,72227,2019,1,Denver,CLARKSON,44 N PEARL ST,80203,1/25/2016,1/25/2016,Active,...,32,2,7.0,5.0,10/31/2018,Validated,Accepted,No,NaN,NaN
2,72301,2019,1,Denver,BRUMER,8101 E MISSISSIPPI AVE,80247,3/12/2018,6/18/2015,Active,...,31,6,5.0,3.0,10/30/2018,Validated,Accepted,No,NaN,NaN
3,72305,2019,1,Denver,STOREY,7101 W YALE AVE,80227,6/26/2015,7/30/2010,Active,...,16,1,2.0,2.0,10/25/2018,Validated,Accepted,No,NaN,NaN
4,72407,2019,1,Denver,STOREY,7101 W YALE AVE,80227,2/5/2016,7/30/2010,Active,...,16,1,2.0,2.0,10/25/2018,Validated,Accepted,No,NaN,NaN


In [0]:
#Check shape and NaN
def nancheck(df):
    print('Shape of the dataframe:',df.shape,'\n')
    print('Missing value counts:')
    print(df.shape[0]-df.count(),'\n') 
    return

nancheck(df_voter_201901)

Shape of the dataframe: (465757, 28) 

Missing value counts:
VOTER_ID                       0
Year                           0
Month                          0
COUNTY                         0
LAST_NAME                     19
Address                        0
RES_ZIP_CODE                   0
EFFECTIVE_DATE                 0
REGISTRATION_DATE              0
STATUS                         0
BIRTH_YEAR                     0
GENDER                         0
PRECINCT                       0
SPLIT                          0
PARTY                          0
PREFERENCE                458197
PARTY_AFFILIATION_DATE         0
CONGRESSIONAL                  0
STATE_SENATE                   0
STATE_HOUSE                    0
COUNCIL_DISTRICT           77221
SCHOOL_DISTRICT            77221
BALLOT_RETURNED_DATE      197218
BALLOT_STAGE               88481
BALLOT_STATUS             197219
IN_PERSON_VOTE             77221
IN_PERSON_VOTE_DATE       454497
VOTED_PARTY               461575
dtype: int64 



In [0]:
# Load Nov 2018.
def clean_address(address_series):
    new_series = address_series.str.split("APT").str[0]
    new_series = new_series.str.split("UNIT").str[0]
    new_series = new_series.str.split("STE").str[0]
    new_series = new_series.str.split("#").str[0]
    new_series = new_series.str.split("RM").str[0]
    new_series = new_series.str.split("PH").str[0]
    new_series = new_series.str.strip()
    return new_series
  
def load_df(year,part):  
    workdir='gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Raw datasets/Colorado_Registered_Voters/'
    filename=workdir+str(year)+'_Registered_Voters_List_ Part'+str(1)+'.txt'
    df = pd.read_csv(filename, dtype={'HOUSE_NUM':str})

    for i in range(2,part+1):
        filename=workdir+str(year)+'_Registered_Voters_List_ Part'+str(i)+'.txt'
        df_temp = pd.read_csv(filename,encoding='ISO-8859-1', dtype={'HOUSE_NUM':str})
        df = pd.concat([df,df_temp])
    df = df[df['COUNTY']=='Denver']
    df['Year']=year
    df['Month']=month
    df.fillna(value='')
    return df

year=2018
month=11
part=8

df=load_df(year, part)
nancheck(df)

df['Address']=clean_address(df['RESIDENTIAL_ADDRESS'])
df = df.sort_values(by=['Address'])
df = df.reset_index()
df.head()

df['CONGRESSIONAL']=df['CONGRESSIONAL'].str.split(" ").str[1]
df['STATE_SENATE']=df['STATE_SENATE'].str.split(" ").str[2]
df['STATE_HOUSE']=df['STATE_HOUSE'].str.split(" ").str[2]

df_voter_201811 = df[['VOTER_ID','Year','Month','COUNTY','LAST_NAME','Address','RESIDENTIAL_ZIP_CODE','EFFECTIVE_DATE','REGISTRATION_DATE','STATUS','BIRTH_YEAR','GENDER','PRECINCT','SPLIT','PARTY','PREFERENCE','PARTY_AFFILIATION_DATE','CONGRESSIONAL','STATE_SENATE','STATE_HOUSE']]
df_voter_201811=df_voter_201811.rename(index=str, columns={'RESIDENTIAL_ZIP_CODE':'RES_ZIP_CODE'})
df_voter_201811.shape

In [0]:
# Load Aug 2017
  
year=2017
month=8
part=8

df=load_df(year, part)
nancheck(df)

df['Address']=clean_address(df['RESIDENTIAL_ADDRESS'])
df = df.sort_values(by=['Address'])
df = df.reset_index()
df.head()

df['CONGRESSIONAL']=df['CONGRESSIONAL'].str.split(" ").str[1]
df['STATE_SENATE']=df['STATE_SENATE'].str.split(" ").str[2]
df['STATE_HOUSE']=df['STATE_HOUSE'].str.split(" ").str[2]

df_voter_201708 = df[['VOTER_ID','Year','Month','COUNTY','LAST_NAME','Address','RESIDENTIAL_ZIP_CODE','EFFECTIVE_DATE','REGISTRATION_DATE','STATUS','BIRTH_YEAR','GENDER','PRECINCT','SPLIT','PARTY','PARTY_AFFILIATION_DATE','CONGRESSIONAL','STATE_SENATE','STATE_HOUSE']]
df_voter_201708=df_voter_201708.rename(index=str, columns={'RESIDENTIAL_ZIP_CODE':'RES_ZIP_CODE'})
df_voter_201708.head()

,VOTER_ID,Year,Month,COUNTY,LAST_NAME,Address,RES_ZIP_CODE,EFFECTIVE_DATE,REGISTRATION_DATE,STATUS,BIRTH_YEAR,GENDER,PRECINCT,SPLIT,PARTY,PARTY_AFFILIATION_DATE,CONGRESSIONAL,STATE_SENATE,STATE_HOUSE
0,200129262,2017,8,Denver,CRANMER,1 E BAYAUD AVE,80209,11/8/2016,11/8/2016,Active,1973,Male,1320216218,218.00,DEM,09/02/2008,1,32,2
1,200187177,2017,8,Denver,BROOKSHIER,1 E BAYAUD AVE,80209,6/28/2016,10/6/2008,Inactive,1963,Male,1320216218,218.00,REP,10/06/2008,1,32,2
2,888600,2017,8,Denver,MACK,1 E BAYAUD AVE,80209,11/8/2016,4/1/2008,Active,1985,Male,1320216218,218.00,UAF,11/04/2011,1,32,2
3,600374777,2017,8,Denver,FISHER,1 E BAYAUD AVE,80209,11/8/2016,10/31/2011,Active,1983,Male,1320216218,218.00,DEM,10/31/2011,1,32,2
4,2679172,2017,8,Denver,WATSON,1 E BAYAUD AVE,80209,11/8/2016,4/4/2000,Active,1960,Male,1320216218,218.00,DEM,04/04/2000,1,32,2


In [0]:
# Load Aug 2016
year=2016
month=8
part=14

df=load_df(year, part)

## df['Address']=clean_address(df['RESIDENTIAL_ADDRESS']) # Can't use this, a different format of street with no space between street number and pre-direction
df['RESIDENTIAL_ADDRESS_New']=df.apply(lambda x:(x.RESIDENTIAL_ADDRESS[:len(str(x.HOUSE_NUM))]+' '+x.RESIDENTIAL_ADDRESS[len(str(x.HOUSE_NUM)):]),axis=1)
df['Address']=clean_address(df['RESIDENTIAL_ADDRESS_New'])
df = df.sort_values(by=['Address'])
df = df.reset_index()

df['CONGRESSIONAL']=df['CONGRESSIONAL'].str.split(" ").str[1]
df['STATE_SENATE']=df['STATE_SENATE'].str.split(" ").str[2]
df['STATE_HOUSE']=df['STATE_HOUSE'].str.split(" ").str[2]

df_voter_201608 = df[['VOTER_ID','Year','Month','COUNTY','LAST_NAME','Address','RESIDENTIAL_ZIP_CODE','EFFECTIVE_DATE','REGISTRATION_DATE','STATUS','BIRTH_YEAR','GENDER','PRECINCT','SPLIT','PARTY','PARTY_AFFILIATION_DATE','CONGRESSIONAL','STATE_SENATE','STATE_HOUSE']]
df_voter_201608=df_voter_201608.rename(index=str, columns={'RESIDENTIAL_ZIP_CODE':'RES_ZIP_CODE'})
df_voter_201608.head()

,VOTER_ID,Year,Month,COUNTY,LAST_NAME,Address,RES_ZIP_CODE,EFFECTIVE_DATE,REGISTRATION_DATE,STATUS,BIRTH_YEAR,GENDER,PRECINCT,SPLIT,PARTY,PARTY_AFFILIATION_DATE,CONGRESSIONAL,STATE_SENATE,STATE_HOUSE
0,600374777,2016,8,Denver,FISHER,1 E BAYAUD AVE,80209,11/4/2014,10/31/2011,Active,1983,Male,1320216218,218.00,DEM,10/31/2011,1,32,2
1,200187177,2016,8,Denver,BROOKSHIER,1 E BAYAUD AVE,80209,10/6/2008,10/6/2008,Active,1963,Male,1320216218,218.00,REP,10/06/2008,1,32,2
2,2628832,2016,8,Denver,GILBERTSON,1 E BAYAUD AVE,80209,10/4/2012,10/26/1998,Active,1969,Male,1320216218,218.00,REP,10/04/2012,1,32,2
3,2565811,2016,8,Denver,MULLINS,1 E BAYAUD AVE,80209,11/5/1996,11/5/1996,Active,1963,Male,1320216218,218.00,REP,11/05/1996,1,32,2
4,600350832,2016,8,Denver,VENEZIA,1 E BAYAUD AVE,80209,11/4/2014,7/13/2011,Active,1983,Male,1320216218,218.00,DEM,07/13/2011,1,32,2


In [0]:
# Load Oct 2015
year=2015
month=10
part=14

df=load_df(year, part)
nancheck(df)

df['Address']=clean_address(df['RESIDENTIAL_ADDRESS'])
df = df.sort_values(by=['Address'])
df = df.reset_index()
df.head()

df['CONGRESSIONAL']=df['CONGRESSIONAL'].str.split(" ").str[1]
df['STATE_SENATE']=df['STATE_SENATE'].str.split(" ").str[2]
df['STATE_HOUSE']=df['STATE_HOUSE'].str.split(" ").str[2]

df_voter_201510 = df[['VOTER_ID','Year','COUNTY','LAST_NAME','Address','RESIDENTIAL_ZIP_CODE','EFFECTIVE_DATE','REGISTRATION_DATE','STATUS','BIRTH_YEAR','GENDER','PRECINCT','SPLIT','PARTY','PARTY_AFFILIATION_DATE','CONGRESSIONAL','STATE_SENATE','STATE_HOUSE']]
df_voter_201510=df_voter_201510.rename(index=str, columns={'RESIDENTIAL_ZIP_CODE':'RES_ZIP_CODE'})
df_voter_201510.head()

,VOTER_ID,Year,COUNTY,LAST_NAME,Address,RES_ZIP_CODE,EFFECTIVE_DATE,REGISTRATION_DATE,STATUS,BIRTH_YEAR,GENDER,PRECINCT,SPLIT,PARTY,PARTY_AFFILIATION_DATE,CONGRESSIONAL,STATE_SENATE,STATE_HOUSE
0,2565811,2015,Denver,MULLINS,1 E BAYAUD AVE,80209,11/5/1996,11/5/1996,Active,1963,Male,1320216218,218.00,REP,11/05/1996,NaN,NaN,NaN
1,600374777,2015,Denver,FISHER,1 E BAYAUD AVE,80209,11/4/2014,10/31/2011,Active,1983,Male,1320216218,218.00,DEM,10/31/2011,NaN,NaN,NaN
2,2679172,2015,Denver,WATSON,1 E BAYAUD AVE,80209,11/4/2014,4/4/2000,Active,1960,Male,1320216218,218.00,DEM,04/04/2000,NaN,NaN,NaN
3,200187177,2015,Denver,BROOKSHIER,1 E BAYAUD AVE,80209,10/6/2008,10/6/2008,Active,1963,Male,1320216218,218.00,REP,10/06/2008,NaN,NaN,NaN
4,600350832,2015,Denver,VENEZIA,1 E BAYAUD AVE,80209,11/4/2014,7/13/2011,Active,1983,Male,1320216218,218.00,DEM,07/13/2011,NaN,NaN,NaN


In [0]:
# Load Sep 2014
year=2014
month=9
part=14

df=load_df(year, part)
nancheck(df)


df['Address']=clean_address(df['RESIDENTIAL_ADDRESS'])
df = df.sort_values(by=['Address'])
df = df.reset_index()
df.head()

#No Congressional data available for 2014.
df_voter_201409 = df[['VOTER_ID','Year','COUNTY','LAST_NAME','Address','RESIDENTIAL_ZIP_CODE','EFFECTIVE_DATE','REGISTRATION_DATE','STATUS','BIRTH_YEAR','GENDER','PRECINCT','SPLIT','PARTY','PARTY_AFFILIATION_DATE']]
df_voter_201409=df_voter_201409.rename(index=str, columns={'RESIDENTIAL_ZIP_CODE':'RES_ZIP_CODE'})
df_voter_201409.head()

,VOTER_ID,Year,COUNTY,LAST_NAME,Address,RES_ZIP_CODE,EFFECTIVE_DATE,REGISTRATION_DATE,STATUS,BIRTH_YEAR,GENDER,PRECINCT,SPLIT,PARTY,PARTY_AFFILIATION_DATE
0,2883371.0,2014,Denver,TORREZ,1 E BAYAUD AVE,80209.0,10/09/2012,10/04/2004,Active,1961.0,Male,1.320216e+09,218.00,DEM,10/04/2004
1,2679172.0,2014,Denver,WATSON,1 E BAYAUD AVE,80209.0,07/26/2012,04/04/2000,Active,1960.0,Male,1.320216e+09,218.00,DEM,04/04/2000
2,200187177.0,2014,Denver,BROOKSHIER,1 E BAYAUD AVE,80209.0,10/06/2008,10/06/2008,Active,1963.0,Male,1.320216e+09,218.00,REP,10/06/2008
3,2628832.0,2014,Denver,GILBERTSON,1 E BAYAUD AVE,80209.0,10/04/2012,10/26/1998,Active,1969.0,Male,1.320216e+09,218.00,REP,10/04/2012
4,2565811.0,2014,Denver,MULLINS,1 E BAYAUD AVE,80209.0,11/05/1996,11/05/1996,Active,1963.0,Male,1.320216e+09,218.00,REP,11/05/1996


In [0]:
#Set checkpoints and merge 2014-2019 datasets
workdir='gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/'

def write_checkpoint(df, dfname):
  filename=workdir+dfname+' '+time.strftime("%Y%m%d-%H%M")+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename)
'''
write_checkpoint(df_voter_201901,'Clean_Denver_voter_201901')  ## 465757 records
write_checkpoint(df_voter_201811,'Clean_Denver_voter_201811')  ## 482094 records
write_checkpoint(df_voter_201708,'Clean_Denver_voter_201708')  ## 445368 records
write_checkpoint(df_voter_201608,'Clean_Denver_voter_201608')  ## 439224 records
write_checkpoint(df_voter_201510,'Clean_Denver_voter_201510')  ## 423516 records
write_checkpoint(df_voter_201409,'Clean_Denver_voter_201409')  ## 464175 records

df_voter_201901=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Clean_Denver_voter_201901.csv')
df_voter_201811=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Clean_Denver_voter_201811.csv')
df_voter_201708=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Clean_Denver_voter_201708.csv')
df_voter_201608=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Clean_Denver_voter_201608.csv')
df_voter_201510=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Clean_Denver_voter_201510.csv')
df_voter_201409=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Clean_Denver_voter_201409.csv')
'''
df_voter=pd.concat([df_voter_201901,df_voter_201811,df_voter_201708,df_voter_201608,df_voter_201510,df_voter_201409])
nancheck(df_voter) ## 2720134 records

Shape of the dataframe: (2720134, 28) 

Missing value counts:
Address                         0
BALLOT_RETURNED_DATE      2451595
BALLOT_STAGE              2342858
BALLOT_STATUS             2451596
BIRTH_YEAR                      0
CONGRESSIONAL              464175
COUNCIL_DISTRICT          2331598
COUNTY                          0
EFFECTIVE_DATE                  0
GENDER                          0
IN_PERSON_VOTE            2331598
IN_PERSON_VOTE_DATE       2708874
LAST_NAME                     102
Month                     1815153
PARTY                           0
PARTY_AFFILIATION_DATE          0
PRECINCT                        0
PREFERENCE                2706194
REGISTRATION_DATE               0
RES_ZIP_CODE                    0
SCHOOL_DISTRICT           2331598
SPLIT                           0
STATE_HOUSE                464175
STATE_SENATE               464175
STATUS                          0
VOTED_PARTY               2715952
VOTER_ID                        0
Year                

In [0]:
#Clean the new dataframe to remove unneeded columns/rows and rename the rest
df_voter_clean=df_voter.dropna(thresh=len(df_voter)*.75, axis=1)
nancheck(df_voter_clean)
df_voter_clean.drop(columns=['COUNTY','PARTY_AFFILIATION_DATE'],inplace=True)
df_voter_clean=df_voter_clean.rename(index=str, columns={'VOTER_ID':'VoterID','LAST_NAME':'VoterLastName','EFFECTIVE_DATE':'EffectiveDate','GENDER':'Gender','BIRTH_YEAR':'BirthYear','PARTY':'Party','PRECINCT':'Precinct','STATUS':'Status','SPLIT':'Split','STATE_HOUSE':'StateHouse','STATE_SENATE':'StateSenate','CONGRESSIONAL':'Congressional'})
df_voter_clean=df_voter_clean.rename(index=str, columns={'REGISTRATION_DATE':'RegistrationDate','RES_ZIP_CODE':'ZIP'})
df_voter_clean.head()

Shape of the dataframe: (2720134, 18) 

Missing value counts:
Address                        0
BIRTH_YEAR                     0
CONGRESSIONAL             464175
COUNTY                         0
EFFECTIVE_DATE                 0
GENDER                         0
LAST_NAME                    102
PARTY                          0
PARTY_AFFILIATION_DATE         0
PRECINCT                       0
REGISTRATION_DATE              0
RES_ZIP_CODE                   0
SPLIT                          0
STATE_HOUSE               464175
STATE_SENATE              464175
STATUS                         0
VOTER_ID                       0
Year                           0
dtype: int64 



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Address,BirthYear,Congressional,EffectiveDate,Gender,VoterLastName,Party,Precinct,RegistrationDate,ZIP,Split,StateHouse,StateSenate,Status,VoterID,Year
0,161 S GROVE ST,1982.0,1,2/5/2017,Female,KING,DEM,1.340416e+09,7/2/2016,80219.0,434,4,34,Active,72085.0,2019
1,44 N PEARL ST,1939.0,1,1/25/2016,Female,CLARKSON,REP,1.320216e+09,1/25/2016,80203.0,251,2,32,Active,72227.0,2019
2,8101 E MISSISSIPPI AVE,1925.0,1,3/12/2018,Male,BRUMER,UAF,1.310617e+09,6/18/2015,80247.0,650,6,31,Active,72301.0,2019
3,7101 W YALE AVE,1954.0,1,6/26/2015,Female,STOREY,REP,1.160116e+09,7/30/2010,80227.0,112,1,16,Active,72305.0,2019
4,7101 W YALE AVE,1954.0,1,2/5/2016,Male,STOREY,REP,1.160116e+09,7/30/2010,80227.0,112,1,16,Active,72407.0,2019


In [0]:
# Merge with other tables.
df_LastName=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Cleaned_SurnameToRaces_Census2010_Full.csv')
df_CensusBlockGroup=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Cleaned_Denver_Address_CensusBlockGroup.csv')
df_LastName['Surname'].astype('str')  ## The 102 NaN value is LAST_NAME actually represent people with surname 'NULL'
df_LastName['Surname'].fillna(value='NULL')
df_CensusBlockGroup=read_checkpoint('gdrive/My Drive/Project/GoCode 2019/Go Code - Regis Voter/Data/Merging Dataframes/Cleaned_Denver_Address_CensusBlockGroup.csv')
df1 = df_voter_clean.merge(df_LastName, how='left', left_on= 'VoterLastName', right_on = 'Surname')
df2 = df1.merge(df_CensusBlockGroup,  how='left', left_on= 'Address', right_on = 'Address')
nancheck(df1) #2720134
###df2.drop(columns=['Month','Surname','Longitude','Latitude'],inplace=True) #2216160
df_final_clean=df2
### df_final_clean=df2.dropna(how='any')
nancheck(df_final_clean)
write_checkpoint(df_final_clean,'Denver_Voters_Master_2014-2019')

In [0]:
df_final_clean[df_final_clean['Year']==2016].head(100)

,Address,BirthYear,Congressional,EffectiveDate,Gender,VoterLastName,Party,Precinct,RegistrationDate,ZIP,...,Surname,PercentWhiteAlone,PercentBlackOrAfricanAmericanAlone,PercentAsianAndNativeHawaiianAndOtherPacificIslanderAlone,PercentAmericanIndianAndAlaskaNativeAlone,Percent2OrMoreRaces,PercentHispanicOrLatinoOrigin,Longitude,Latitude,Tract-CensusBlock
1393219,1 E BAYAUD AVE,1983.0,1,11/4/2014,Male,FISHER,DEM,1.320216e+09,10/31/2011,80209.0,...,FISHER,82.58,11.790,0.550,0.760,1.890,2.430,-104.987145,39.714908,1.080310e+12
1393220,1 E BAYAUD AVE,1963.0,1,10/6/2008,Male,BROOKSHIER,REP,1.320216e+09,10/6/2008,80209.0,...,BROOKSHIER,89.85,0.960,1.150,0.960,2.300,4.790,-104.987145,39.714908,1.080310e+12
1393221,1 E BAYAUD AVE,1969.0,1,10/4/2012,Male,GILBERTSON,REP,1.320216e+09,10/26/1998,80209.0,...,GILBERTSON,95.68,0.350,0.780,0.290,1.240,1.660,-104.987145,39.714908,1.080310e+12
1393222,1 E BAYAUD AVE,1963.0,1,11/5/1996,Male,MULLINS,REP,1.320216e+09,11/5/1996,80209.0,...,MULLINS,88.75,6.860,0.360,0.520,1.780,1.740,-104.987145,39.714908,1.080310e+12
1393223,1 E BAYAUD AVE,1983.0,1,11/4/2014,Male,VENEZIA,DEM,1.320216e+09,7/13/2011,80209.0,...,VENEZIA,94.01,0.350,0.670,0.210,0.990,3.760,-104.987145,39.714908,1.080310e+12
1393224,1 E BAYAUD AVE,1960.0,1,11/4/2014,Male,WATSON,DEM,1.320216e+09,4/4/2000,80209.0,...,WATSON,66.03,27.900,0.530,0.820,2.300,2.410,-104.987145,39.714908,1.080310e+12
1393225,1 N ASH ST,1917.0,1,7/14/2013,Female,FREED,REP,1.310617e+09,5/1/1947,80220.0,...,FREED,94.86,1.170,0.780,0.360,1.110,1.720,-104.938746,39.716458,1.080310e+12
1393226,1 N ASH ST,1950.0,1,7/14/2008,Female,FREED,DEM,1.310617e+09,3/5/1999,80220.0,...,FREED,94.86,1.170,0.780,0.360,1.110,1.720,-104.938746,39.716458,1.080310e+12
1393227,1 N BELLAIRE ST,1962.0,1,4/4/2011,Female,CHAFAI,REP,1.310617e+09,1/23/2003,80220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-104.937644,39.716428,1.080310e+12
1393228,1 N BROADWAY ST,1986.0,1,11/4/2014,Male,LAWSON,REP,1.320216e+09,12/2/2013,80203.0,...,LAWSON,73.84,20.740,0.430,0.650,2.150,2.200,NaN,NaN,NaN
